# axerunt

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import reduce
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('axerunt_test.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
from dask import delayed

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, is_valid, is_type, null_fn, identity_fn, pd_abs, pd_is_empty, ser_range_center_clip
from common_util import str_now_dtz, last_commit_dtz, load_json, dump_json, pd_slot_shift, arr_nonzero, get_variants, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import GRAPHS_DIR, HISTORY_DIR, TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import get_rm_keychain, process_transform
from runt_util import RUNT_TYPE_MAPPING, RUNTFormatError, get_ser_fn

DEBUG:matplotlib:$HOME=/home/kev
DEBUG:matplotlib:CONFIGDIR=/home/kev/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.0
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'impo

CRITICAL:root:script location: /home/kev/crunch/mutate/axerunt_test.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/
DEBUG:matplotlib:CACHEDIR=/home/kev/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kev/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## Init DataAPI

In [2]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
graphs, transforms = get_graphs(), get_transforms()

## View

In [5]:
data_rcs, data_dfs = DataAPI.axe_load(['dohlca', 'dohlca'])

In [9]:
data_dfs[list(data_dfs.keys())[0]]

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
id,,,,,
2009-01-02 00:00:00+00:00,8771.77,9065.28,8761.10,9034.69,8934.514025
2009-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2009-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2009-01-05 00:00:00+00:00,9027.13,9033.66,8892.68,8952.89,8964.985225
2009-01-06 00:00:00+00:00,8954.57,9087.82,8941.11,9015.10,9009.688837
2009-01-07 00:00:00+00:00,8996.94,8996.94,8720.00,8769.70,8820.237200
2009-01-08 00:00:00+00:00,8769.94,8769.94,8651.27,8742.46,8700.459338
2009-01-09 00:00:00+00:00,8738.80,8769.07,8577.27,8599.18,8640.200900
2009-01-10 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
